In [1]:
import sys, os; sys.path.append('..')
import random
import itertools
import pyzx as zx

from fractions import Fraction
%config InlineBackend.figure_format = 'svg'
import numpy as np
import math

In [2]:
random.seed(1337)
c0 = zx.generate.CNOT_HAD_PHASE_circuit(20, 200)
print(c0.stats())
g = c0.to_graph()
zx.full_reduce(g)
g.normalize()
#zx.draw(g1)

Circuit  on 20 qubits with 200 gates.
        49 is the T-count
        151 Cliffords among which 
        117 2-qubit gates (117 CNOT, 0 other) and
        34 Hadamard gates.


In [10]:
k = list(g.neighbors(12))
k.remove(143)
k

[]

In [3]:
zx.draw(g, labels=True)

In [4]:
def twoq_score(g):
    c = zx.extract_circuit(g.copy())
    return c.twoqubitcount()

In [5]:
random.seed(1337)
print(twoq_score(g))
g1 = zx.sparsify.pivot_anneal(g, 1000, score=twoq_score)
print(twoq_score(g1))

383
383 at iteration 1 ^
363 at iteration 2 v
352 at iteration 4 v
329 at iteration 6 v
348 at iteration 106 ^
352 at iteration 111 ^
353 at iteration 112 ^
361 at iteration 119 ^
355 at iteration 121 v
345 at iteration 122 v
338 at iteration 126 v
338 at iteration 127 ^
334 at iteration 128 v
324 at iteration 130 v
325 at iteration 143 ^
340 at iteration 175 ^
342 at iteration 176 ^
326 at iteration 177 v
358 at iteration 182 ^
376 at iteration 207 ^
378 at iteration 209 ^
381 at iteration 213 ^
376 at iteration 214 v
375 at iteration 227 v
377 at iteration 228 ^
368 at iteration 231 v
374 at iteration 243 ^
362 at iteration 246 v
346 at iteration 257 v
342 at iteration 336 v
342 at iteration 359 ^
360 at iteration 459 ^
360 at iteration 462 ^
345 at iteration 468 v
351 at iteration 570 ^
353 at iteration 571 ^
342 at iteration 574 v
343 at iteration 575 ^
342 at iteration 595 v
340 at iteration 608 v
338 at iteration 612 v
342 at iteration 614 ^
348 at iteration 621 ^
321 at iteratio

In [7]:
g_fr = g.copy()
g_fr_sparse = g1.copy()

c1 = zx.extract_circuit(g_fr)
c2 = zx.extract_circuit(g_fr_sparse)
c1count = c1.twoqubitcount()
c2count = c2.twoqubitcount()
print("original", c1count)
print("sparsified", c2count)
print("reduction", (c1count - c2count)/c1count)

original 383
sparsified 311
reduction 0.18798955613577023


In [4]:
zx.draw(g1)

In [5]:
print(c1.stats())
print(c2.stats())

Circuit  on 9 qubits with 2040 gates.
        285 is the T-count
        1755 Cliffords among which 
        1361 2-qubit gates and 277 Hadamard gates.
Circuit  on 9 qubits with 1709 gates.
        289 is the T-count
        1420 Cliffords among which 
        1139 2-qubit gates and 219 Hadamard gates.


In [6]:
print("c1",
 sum(1 if isinstance(g,zx.circuit.CNOT) else 0 for g in c1.gates),
 sum(1 if isinstance(g,zx.circuit.CZ) else 0 for g in c1.gates)
)
print("c2",
 sum(1 if isinstance(g,zx.circuit.CNOT) else 0 for g in c2.gates),
 sum(1 if isinstance(g,zx.circuit.CZ) else 0 for g in c2.gates)
)

c1 928 433
c2 912 227


In [6]:
arch = Architecture(name="9Q", coupling_matrix=np.array([
    [0, 1, 0, 0, 0, 1, 0, 0, 0],
    [1, 0, 1, 0, 1, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0, 0, 1],
    [0, 1, 0, 1, 0, 1, 0, 1, 0],
    [1, 0, 0, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 1, 0, 0, 0, 1, 0]
]))

In [7]:
zx.draw(g1)

In [22]:
(sum(1 if isinstance(g,zx.circuit.CZ) else 0 for g in c.gates),
 sum(1 if isinstance(g,zx.circuit.CNOT) else 0 for g in c.gates))

(360, 1848)

In [7]:
zx.draw(g1)

In [28]:
m1 = m.copy()
ops = zx.Mat2.id(9)
m1.gauss(full_reduce=True, x=ops)
for i in range(9):
    if not i in [0,5,8]: ops[i,:] = unitv(9,i).transpose()
m1

[ 1  0  0  0  0  0  0  0  0  0  0  0  0 ]
[ 0  1  0  0  0  1  0  1  0  0  0  0  1 ]
[ 0  0  1  0  0  1  0  0  0  0  0  0  0 ]
[ 0  0  0  1  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  1  1  0  0  0  0  1  0  1 ]
[ 0  0  0  0  0  0  1  0  0  0  0  0  0 ]
[ 0  0  0  0  0  0  0  0  1  0  1  0  0 ]
[ 0  0  0  0  0  0  0  0  0  1  0  0  0 ]
[ 0  0  0  0  0  0  0  0  0  0  0  1  0 ]

In [48]:
ops

[ 1  1  0  1  0  0  0  0  1 ]
[ 0  1  0  0  0  0  0  0  0 ]
[ 0  0  1  0  0  0  0  0  0 ]
[ 0  0  0  1  0  0  0  0  0 ]
[ 0  0  0  0  1  0  0  0  0 ]
[ 1  0  0  0  0  1  1  0  1 ]
[ 0  0  0  0  0  0  1  0  0 ]
[ 0  0  0  0  0  0  0  1  0 ]
[ 1  0  1  1  1  0  0  0  0 ]

In [53]:
ops1 = ops.copy()
tr = CNOT_tracker(9)
ops1.gauss(full_reduce=True, y=tr,blocksize=3)

9

In [54]:
len(tr.gates)

11

In [56]:
tr.gates

[CNOT(1,0),
 CNOT(3,0),
 CNOT(8,0),
 CNOT(6,5),
 CNOT(3,5),
 CNOT(4,8),
 CNOT(2,8),
 CNOT(1,8),
 CNOT(1,5),
 CNOT(0,8),
 CNOT(0,5)]

In [55]:
tr.matrix * m

[ 1  0  0  0  0  0  0  0  0  0  0  0  0 ]
[ 1  0  1  0  0  1  0  0  1  0  1  0  0 ]
[ 0  0  0  1  1  1  0  0  0  1  1  1  0 ]
[ 1  1  1  0  0  0  1  1  0  1  0  0  1 ]
[ 1  0  0  1  0  0  1  0  1  1  1  1  1 ]
[ 0  0  0  0  0  0  1  0  0  0  0  0  0 ]
[ 0  1  1  0  0  0  0  1  0  0  0  0  1 ]
[ 0  0  1  1  0  1  1  0  1  1  1  0  1 ]
[ 0  0  0  0  0  0  0  0  0  0  0  1  0 ]

In [14]:
def unitv(d,i):
    return zx.Mat2([[1 if i == j else 0] for j in range(d)])

In [16]:

ops

[ 1  0  0  1  0  0  0  0  1 ]
[ 0  1  1  0  0  0  1  1  1 ]
[ 0  0  1  0  0  0  0  1  1 ]
[ 0  0  0  1  0  0  1  1  0 ]
[ 1  0  0  0  1  1  1  1  0 ]
[ 1  0  0  0  0  1  1  0  1 ]
[ 1  0  1  1  0  0  0  1  0 ]
[ 0  0  0  0  0  1  0  0  0 ]
[ 1  0  1  1  1  0  0  0  0 ]

In [16]:
g1.normalize()
zx.draw(g1)